# Capitulo 05 Escrita Dados S3

Notebook gerado automaticamente a partir do código fonte python.


In [1]:
import duckdb
import os
import boto3
from botocore.exceptions import ClientError

# ==============================================================================
# SETUP MINIO
# ==============================================================================
print(f"--- Iniciando Capítulo 05: Escrita de Dados S3 ---")

MINIO_ENDPOINT = "http://localhost:9000"
MINIO_ACCESS_KEY = "admin"
MINIO_SECRET_KEY = "password"
BUCKET_NAME = "learn-duckdb-s3"

s3_client = boto3.client(
    's3',
    endpoint_url=MINIO_ENDPOINT,
    aws_access_key_id=MINIO_ACCESS_KEY,
    aws_secret_access_key=MINIO_SECRET_KEY
)

try:
    s3_client.create_bucket(Bucket=BUCKET_NAME)
    print(f"Bucket {BUCKET_NAME} pronto.")
except ClientError:
    pass

# ==============================================================================
# EXEMPLOS DUCKDB
# ==============================================================================
con = duckdb.connect(database=':memory:')
con.execute("INSTALL httpfs; LOAD httpfs;")
con.execute(f"""
CREATE SECRET secret_minio (
    TYPE S3,
    KEY_ID '{MINIO_ACCESS_KEY}',
    SECRET '{MINIO_SECRET_KEY}',
    REGION 'us-east-1',
    ENDPOINT '{MINIO_ENDPOINT.replace("http://", "")}',
    URL_STYLE 'path',
    USE_SSL 'false'
);
""")

# 1. Criar dados locais
print("\n--- Gerando dados locais ---")
con.execute("""
CREATE TABLE vendas AS
SELECT
    range as id,
    'Produto_' || (range % 5) as produto,
    (random() * 100)::DECIMAL(10,2) as valor
FROM range(10);
""")
print(con.execute("SELECT * FROM vendas LIMIT 5").df())

--- Iniciando Capítulo 05: Escrita de Dados S3 ---

--- Gerando dados locais ---
   id    produto  valor
0   0  Produto_0  19.74
1   1  Produto_1  29.25
2   2  Produto_2  32.14
3   3  Produto_3  53.05
4   4  Produto_4  41.96


In [2]:
# 2. Escrever Parquet
print("\n--- Escrevendo Parquet no S3 ---")
con.execute(f"COPY vendas TO 's3://{BUCKET_NAME}/output_vendas.parquet' (FORMAT PARQUET)")
print("Arquivo parquet escrito com sucesso.")

# 3. Escrever CSV (com opções)
print("\n--- Escrevendo CSV no S3 ---")
con.execute(f"""
COPY vendas TO 's3://{BUCKET_NAME}/output_vendas.csv' 
(FORMAT CSV, DELIMITER ';', HEADER TRUE)
""")
print("Arquivo CSV escrito com sucesso.")

# 4. Escrever JSON
print("\n--- Escrevendo JSON no S3 ---")
con.execute(f"""
COPY vendas TO 's3://{BUCKET_NAME}/output_vendas.json' 
(FORMAT JSON)
""")
print("Arquivo JSON escrito com sucesso.")

# 5. Escrever particionado (Hive Partitioning)
print("\n--- Escrevendo Particionado (Hive) ---")
con.execute(f"""
COPY vendas TO 's3://{BUCKET_NAME}/vendas_partitioned' 
(FORMAT PARQUET, PARTITION_BY (produto), OVERWRITE_OR_IGNORE)
""")
print("Dados particionados escritos.")

print("\n--- Verificando Arquivos Criados ---")
files = con.execute(f"SELECT file FROM glob('s3://{BUCKET_NAME}/**')").df()
print(files)


--- Escrevendo Parquet no S3 ---
Arquivo parquet escrito com sucesso.

--- Escrevendo CSV no S3 ---
Arquivo CSV escrito com sucesso.

--- Escrevendo JSON no S3 ---
Arquivo JSON escrito com sucesso.

--- Escrevendo Particionado (Hive) ---


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Dados particionados escritos.

--- Verificando Arquivos Criados ---
                                                 file
0                      s3://learn-duckdb-s3/creds.csv
1                       s3://learn-duckdb-s3/data.csv
2                      s3://learn-duckdb-s3/data.json
3              s3://learn-duckdb-s3/output_vendas.csv
4             s3://learn-duckdb-s3/output_vendas.json
5          s3://learn-duckdb-s3/output_vendas.parquet
6                  s3://learn-duckdb-s3/sales.parquet
7                       s3://learn-duckdb-s3/test.csv
8   s3://learn-duckdb-s3/vendas_partitioned/produt...
9   s3://learn-duckdb-s3/vendas_partitioned/produt...
10  s3://learn-duckdb-s3/vendas_partitioned/produt...
11  s3://learn-duckdb-s3/vendas_partitioned/produt...
12  s3://learn-duckdb-s3/vendas_partitioned/produt...
